In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import ssl
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pytorch_dataset import HatsDataset
import pickle

In [2]:
in_channel = 3
num_class = 15
learning_rate = 1e-3
batch_size = 20
num_epochs = 1
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.RandomCrop(256,padding=3,padding_mode='constant'),
     transforms.RandomHorizontalFlip(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


dataset = HatsDataset(csv_file='HatsOnly.csv',root_dir='HatsOnly',
    transform=transform) #8780

humanset = HatsDataset(csv_file='HumanOnly.csv',root_dir='HumanOnly',
    transform=transform) 

trainset, testset = torch.utils.data.random_split(dataset,[6000,1138])

trainset_human, testset_human = torch.utils.data.random_split(humanset,[1312,327])

humanloader =DataLoader(testset_human, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

trainloader =DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

trainset_mix = torch.utils.data.ConcatDataset([trainset,trainset_human])
testset_mix = torch.utils.data.ConcatDataset([testset,testset_human])

trainloader_mix = DataLoader(trainset_mix, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testloader_mix = DataLoader(testset_mix, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

classes = ('baseballcap', 'BikeHelmet', 'BucketHat', 'CowboyHat',
           'FeltHat', 'FireFighterHat', 'FlatCap', 'GraduationCap', 'Heaterhat', 'MilitaryHelmet',
           'MotorCycle Helmet','Police Hat','SateftyHelmet','TopHat','beanie')


In [3]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


trainloader = DeviceDataLoader(trainloader,'cuda')
testloader = DeviceDataLoader(testloader,'cuda')
humanloader = DeviceDataLoader(humanloader,'cuda')


trainloader_mix = DeviceDataLoader(trainloader_mix,'cuda')
testloader_mix = DeviceDataLoader(testloader_mix,'cuda')

In [4]:
def conv2dModuel(in_channel,out,K_sz=3,pad=1,st=1):
    hidden_layer=[nn.Conv2d(in_channel,out,kernel_size=K_sz,padding=pad,stride=st),
                nn.BatchNorm2d(out),
                nn.LeakyReLU(inplace=True)]
    return nn.Sequential(*hidden_layer)



class ResNetBlock(nn.Module):
    def __init__(self,in_channels,out_channels,connect=True):
        super(ResNetBlock, self).__init__()
        self.connect = connect
        self.conv1 = nn.Sequential(conv2dModuel(in_channels,out_channels),
                                   nn.MaxPool2d(kernel_size=2, stride=2), #downsize
                                   conv2dModuel(out_channels,out_channels))
        self.conv2 = nn.Sequential(conv2dModuel(out_channels,out_channels),
                                   conv2dModuel(out_channels,out_channels))
        self.relu = nn.LeakyReLU(inplace=True)
    def forward(self,x):
        if self.connect==True:
            y = self.conv1(x)
        else:
            y = self.conv2(x)
            y = y+x    
        z = self.conv2(y)
        z = z+y
        z = self.relu(z)
        return z
        
        


class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.conv1 = conv2dModuel(3,64,K_sz=7,pad=3,st=2)
        self.conv2 = ResNetBlock(64,64,connect=False)
        self.conv3 = ResNetBlock(64,128)
        self.conv4 = ResNetBlock(128,256)
        self.conv5 = ResNetBlock(256,512)
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                       nn.Flatten(),
                                       nn.Linear(512*4*4,256),
                                       nn.LeakyReLU(),
                                       nn.Linear(256,128),
                                       nn.LeakyReLU(),
                                       nn.Linear(128,15)
                                       )
        
    def forward(self, x):
  
        x = self.conv1(x) #layer 1
        x = self.conv2(x) #layer 2,3,4,5
        x = self.conv3(x) #layer 6,7,8,9
        x = self.conv4(x) #layer 10,11,12,13
        x = self.conv5(x) #layer 14,15,16,17
        x = self.classifier(x) #(18)
        return x
        #y = self.fc4(x)
        # YOUR CODE HERE
        #raise NotImplementedError()

In [5]:


net = ResNet18().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler()
# comment this line of code if not on gpu
from tqdm.notebook import tqdm
losses = []
for epoch in tqdm(range(50)):  # loop over the dataset multiple times
    loss_temp=0.0      
    for inputs,label in trainloader_mix:
        with torch.cuda.amp.autocast():
             outputs = net(inputs)
             loss = criterion(outputs,label.cuda())
        #outputs = net(inputs)
        #loss = criterion(outputs,label)
        scaler.scale(loss).backward()
        #loss.backward()
        #optimizer.step() 
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        loss_temp += loss.item()    
    losses = np.append(losses,loss_temp)
    # YOUR CODE HERE
plt.plot(losses)
    #raise NotImplementedError()

  0%|          | 0/50 [00:00<?, ?it/s]